In [539]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
%matplotlib inline

In [540]:
names = ['API','Surf_X','Surf_Y','Date_Drilling','Date_Completion','Date_Production','Lateral_Length','Depth_TVD_PPLS','Erosion_PPLS','Pressure_PPLS','TOC_PPLS','Vcarb_PPLS','Vsand_PPLS','Vclay_PPLS','PR_PPLS','YM_PPLS','RHOB_PPLS','Res_PPLS','GR_PPLS','DT_PPLS','DTs_PPLS','Temperature','Temp_Anomaly','S3Tect_PPLS','S3_contrast_PPLS','Heat_Flow','Zone','Nbr_Stages','Frac_Gradient','Proppant_Designed','Proppant_in_Formation','Avg_Breakdown_Pressure','Avg_Treating_Pressure','Max_Treating_pressure','Min_Treating_Pressure','Avg_Rate_Slurry','Max_Rate_Slurry','Min_Rate_Slurry','ShutInPressure_Fil','ShutInPressure_Initial','ISIP','Shot_Density','Shot_Total','Proppant_per_ft','Stage_Spacing','GasCum360','OilCum360']

df_data = pd.read_csv('./TrainSample.csv', 
                        header = None, 
                        sep = ';',
                        decimal = ',',
                        names = names,
                        skiprows = 1,
                        parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                        dtype = {col: np.float32 for col in names}
                       )

df_test = pd.read_csv('./TestSample.csv',
                      header = None, 
                      sep = ';', 
                      decimal = ',', 
                      names = names,
                      skiprows = 1,
                      parse_dates = ['Date_Drilling','Date_Completion','Date_Production'],
                      dtype = {col: np.float32 for col in names}
                     )

#df_data.set_index('API')

df_test.head(5)

,API,Surf_X,Surf_Y,Date_Drilling,Date_Completion,Date_Production,Lateral_Length,Depth_TVD_PPLS,Erosion_PPLS,Pressure_PPLS,...,Min_Rate_Slurry,ShutInPressure_Fil,ShutInPressure_Initial,ISIP,Shot_Density,Shot_Total,Proppant_per_ft,Stage_Spacing,GasCum360,OilCum360
0,633.0,0.201275,-0.577329,2013-06-23,2013-10-26,2014-10-01,-0.694826,0.641631,-0.356227,1.410748,...,0.667402,NaN,-0.740473,0.131374,0.187082,2.409201,0.160140,-0.710739,NaN,NaN
1,587.0,0.222609,0.670307,2012-02-06,2012-03-07,2013-05-08,-0.479516,-1.377997,-1.812483,1.605136,...,0.285738,-0.919600,-1.103957,-1.282464,0.667339,-0.560750,-1.149666,2.085649,NaN,NaN
2,264.0,0.022105,0.404387,2013-05-19,2013-07-24,2013-06-10,0.310782,-0.775745,0.243462,-0.136014,...,0.459046,0.614126,1.029028,1.103339,-0.773433,0.924226,-0.585869,-0.137427,NaN,NaN
3,680.0,0.384359,0.372793,2012-08-18,2012-12-19,2013-12-06,-0.624508,-1.082974,-1.922374,1.443630,...,0.332100,-0.782232,0.450368,-0.136076,0.667339,-0.560750,0.595622,1.887260,NaN,NaN
4,443.0,-0.138513,0.408803,2013-11-09,2014-03-29,2014-11-05,0.338785,-0.616106,0.556826,-1.011425,...,0.323490,NaN,0.743256,0.835687,-1.253690,0.181738,0.151633,-0.216512,NaN,NaN


In [541]:
data_mean_gas = df_data[['GasCum360']].mean()
data_mean_oil = df_data[['OilCum360']].mean()

In [542]:
##Définition des labels

y_gas = {'GasCum360' : df_data['GasCum360']}
y_oil = {'OilCum360' : df_data['OilCum360']}

df_target_gas = pd.DataFrame(y_gas)
df_target_oil = pd.DataFrame(y_oil)

In [543]:
cols_date = ['Date_Drilling','Date_Completion','Date_Production', 'GasCum360', 'OilCum360']

df_data.drop(cols_date, 1, inplace=True)
df_test.drop(cols_date, 1, inplace=True)

In [544]:
df_data.dropna()
df_test.dropna()
df_data = df_data.fillna(df_data.mean())
df_test = df_test.fillna(df_data.mean())

In [545]:
from sklearn import cross_validation

X_train, X_test, y_train_gas, y_test_gas = cross_validation.train_test_split(df_data, df_target_gas, test_size=0.2, random_state=0)
X_train, X_test, y_train_oil, y_test_oil = cross_validation.train_test_split(df_data, df_target_oil, test_size=0.2, random_state=0)

In [546]:
## Fit regression model training

regr_gas = ElasticNet(alpha=0.02, l1_ratio=0.5, max_iter=10)
regr_oil = ElasticNet(alpha=0.018, l1_ratio=0.7, max_iter=12)

clf_gas = MultiOutputRegressor(regr_gas)
clf_oil = MultiOutputRegressor(regr_oil)

clf_gas.fit(X_train, y_train_gas)
clf_oil.fit(X_train, y_train_oil)

y_predict_gas = clf_gas.predict(X_test)
y_predict_oil = clf_oil.predict(X_test)

In [547]:
## Metrics training

retour = 'Erreur Moyenne absolue : {}\n\
Erreur Moyenne carré : {}\n\
R2 : {}'.format(mean_absolute_error(y_test_oil, y_predict_oil),
                mean_squared_error(y_test_oil, y_predict_oil),
                r2_score(y_test_oil, y_predictSVR_oil)               
               )

print(retour)

Erreur Moyenne absolue : 0.41657730937
Erreur Moyenne carré : 0.329201519489
R2 : 0.614449108517


In [548]:
## Metrics training

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

retour = 'Erreur Moyenne absolue : {}\n\
Erreur Moyenne carré : {}\n\
R2 : {}'.format(mean_absolute_error(y_test_gas, y_predict_gas),
                mean_squared_error(y_test_gas, y_predict_gas),
                r2_score(y_test_gas, y_predictSVR_gas),
               )

print(retour)

Erreur Moyenne absolue : 0.433211833239
Erreur Moyenne carré : 0.30500203371
R2 : 0.610129012709


In [549]:
## Fit regression model test data

clf_gas.fit(df_data, df_target_gas)
clf_oil.fit(df_data, df_target_oil)

y_predictFin_gas = clf_gas.predict(df_test)
y_predictFin_oil = clf_oil.predict(df_test)

In [550]:
gascumpred = []
oilcumpred = []

gascumpred = [element[0] for element in y_predictFin_gas]
oilcumpred = [element[0] for element in y_predictFin_oil]

gascumpred_mean = mean_absolute_error(y_test_gas, y_predict_gas)
oilcumpred_mean = mean_absolute_error(y_test_oil, y_predict_oil)

GasCum360_inf = gascumpred - abs(gascumpred_mean)
GasCum360_sup = gascumpred + abs(gascumpred_mean)

OilCum360_inf = oilcumpred - abs(oilcumpred_mean)
OilCum360_sup = oilcumpred + abs(oilcumpred_mean)

GasCum360_inf.tolist
GasCum360_sup.tolist

OilCum360_inf.tolist
OilCum360_sup.tolist

<function tolist>

In [551]:
y_predict_gas[:10]

array([[-0.65601987],
       [ 0.9832021 ],
       [-0.28373086],
       [-1.01154137],
       [-0.87199259],
       [ 0.43868348],
       [-0.20407546],
       [-1.36266661],
       [-0.20417777],
       [-0.72092003]], dtype=float32)

In [552]:
y_test_gas[:10]

,GasCum360
329,-1.056089
153,-0.036146
135,-0.150183
78,-0.798361
452,-0.990523
71,-0.842177
284,0.419826
220,-0.879051
339,-0.338057
247,-0.605402


In [553]:
## Output

id_test = df_test['API'].values.tolist()

output = pd.DataFrame({'API': id_test,
                       'GasCum360_INF': GasCum360_inf,
                       'GasCum360_SUP': GasCum360_sup,
                       'OilCum360_INF': OilCum360_inf,
                       'OilCum360_SUP': OilCum360_sup},
                      index=id_test
                     )

output.head()

output.to_csv('coche-julien-challenge-total.csv', index=False, sep= ';', decimal=',')

In [554]:
print(output.head(5))

         API  GasCum360_INF  GasCum360_SUP  OilCum360_INF  OilCum360_SUP
633.0  633.0      -0.988542      -0.122118      -0.076995       0.756160
587.0  587.0      -0.075915       0.790508      -1.478378      -0.645223
264.0  264.0       0.308100       1.174524      -0.779223       0.053931
680.0  680.0       0.046303       0.912727      -1.272451      -0.439296
443.0  443.0       0.158775       1.025199      -0.404841       0.428314
